# BentoML Example:  Deploy to AWS SageMaker

[BentoML](http://bentoml.ai) is an open source framework for building, shipping and running machine learning services. It provides high-level APIs for defining an ML service and packaging its artifacts, source code, dependencies, and configurations into a production-system-friendly format that is ready for deployment.

This notebook demonstrates how to use BentoML to deploy a machine learning model as a REST API endpoint to AWS SageMaker. For this demo, we are using the [Sentiment Analysis with Scikit-learn](https://github.com/bentoml/BentoML/blob/master/examples/sklearn-sentiment-clf/sklearn-sentiment-clf.ipynb) example, using dataset from [Sentiment140](http://help.sentiment140.com/for-students/).

![Impression](https://www.google-analytics.com/collect?v=1&tid=UA-112879361-3&cid=555&t=event&ec=nb&ea=open&el=official-example&dt=deploy-with-sagemaker)

In [ ]:
!pip install -I bentoml
!pip install sklearn pandas numpy

In [1]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score, roc_curve
from sklearn.pipeline import Pipeline

import bentoml

# Prepare Dataset

In [2]:
%%bash

if [ ! -f ./trainingandtestdata.zip ]; then
    wget -q http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip
    unzip -n trainingandtestdata.zip
fi

In [3]:
columns = ['polarity', 'tweetid', 'date', 'query_name', 'user', 'text']
dftrain = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header = None,
                      encoding ='ISO-8859-1')
dftest = pd.read_csv('testdata.manual.2009.06.14.csv',
                     header = None,
                     encoding ='ISO-8859-1')
dftrain.columns = columns
dftest.columns = columns

# Model Training

In [4]:
sentiment_lr = Pipeline([
                         ('count_vect', CountVectorizer(min_df = 100,
                                                        ngram_range = (1,1),
                                                        stop_words = 'english')), 
                         ('lr', LogisticRegression())])
sentiment_lr.fit(dftrain.text, dftrain.polarity)

/usr/local/anaconda3/envs/dev-py3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
     steps=[('count_vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=100,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
...penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))])

In [5]:
Xtest, ytest = dftest.text[dftest.polarity!=2], dftest.polarity[dftest.polarity!=2]
print(classification_report(ytest,sentiment_lr.predict(Xtest)))

              precision    recall  f1-score   support

           0       0.85      0.80      0.83       177
           4       0.82      0.86      0.84       182

   micro avg       0.83      0.83      0.83       359
   macro avg       0.83      0.83      0.83       359
weighted avg       0.83      0.83      0.83       359



In [6]:
sentiment_lr.predict([Xtest[0]])

array([4])

# Define ML Service with BentoML

In [7]:
%%writefile sentiment_lr_model.py
import pandas as pd
import bentoml
from bentoml.artifact import PickleArtifact
from bentoml.handlers import DataframeHandler

@bentoml.artifacts([PickleArtifact('sentiment_lr')])
@bentoml.env(pip_dependencies=["scikit-learn", "pandas"])
class SentimentLRModel(bentoml.BentoService):
    
    @bentoml.api(DataframeHandler, typ='series')
    def predict(self, series):
        """
        predict expects pandas.Series as input
        """        
        return self.artifacts.sentiment_lr.predict(series)

Overwriting sentiment_lr_model.py


# Save BentoML service archive

In [8]:
from sentiment_lr_model import SentimentLRModel

# Initialize bentoML model with artifacts

bento_model = SentimentLRModel.pack(
    sentiment_lr=sentiment_lr
)

# Save bentoML model to directory
saved_path = bento_model.save("/tmp/bento")

# print the directory containing exported model archive (prefixed with model name and version)
print(saved_path)

[2019-08-17 11:35:22,976] INFO - Searching for dependant modules of sentiment_lr_model:/Users/bozhaoyu/src/bento/examples/deploy-with-sagemaker/sentiment_lr_model.py
[2019-08-17 11:35:44,110] INFO - Copying local python module '/Users/bozhaoyu/src/bento/examples/deploy-with-sagemaker/sentiment_lr_model.py'
[2019-08-17 11:35:44,112] INFO - Done copying local python dependant modules
[2019-08-17 11:35:44,245] INFO - BentoService SentimentLRModel:2019_08_17_c1d76ad6 saved to /tmp/bento/SentimentLRModel/2019_08_17_c1d76ad6
/tmp/bento/SentimentLRModel/2019_08_17_c1d76ad6


# Load BentoML Service from archive

In [ ]:
import bentoml

# Load exported bentoML model archive from path
bento_model = bentoml.load(saved_path)

# Call predict on the restored sklearn model
bento_model.predict(pd.Series(["hello", "hi"]))

*For demo purpurse, copy generated model to ./model folder

In [ ]:
import os
import shutil
shutil.rmtree('./model', ignore_errors=True)
shutil.copytree(saved_path, './model')

# Deploy to AWS SageMaker

```bash
bentoml deploy ARCHIVE_PATH --platform=PLATFORM --region=REGION --stage=STAGE
```
### Arguments:
* archive_path: The file path or s3 that contains BentoML bundles.

### Options:
* platform: REQUIRED.  The platform that you want to deploy bentoml bundle to.  For serverless, we support aws-lambda, aws-lambda-py2 and gcp-function.
* region: OPTIONAL The cloud provider's region you want to deploy in.
* stage: OPTIONAL Stage is a helper value that identify the position in the workflow process. DEFAULT value is dev


In [ ]:
!bentoml deploy ./model --platform aws-sagemaker --region us-west-2

### Make request against the endpoint

We are using AWS CLI to make prediction request with SageMaker endpoint. The result will stored in `output.json` file.

```bash
aws sagemaker-runtime invoke-endpoint \
--endpoint-name SentimentLRModel \
--body '["new food", "bad movie", "chicken nuggets", "good family time"]' \
--content-type "application/json"\
output.json
```

In [ ]:
!aws sagemaker-runtime invoke-endpoint \
--endpoint-name SentimentLRModel \
--body '["new food", "bad movie", "chicken nuggets", "good family time"]' \
--content-type "application/json" \
output.json
!cat output.json

# Check deployment status

```
bentoml checkt-deployment-status ARCHIVE_PATH --platform=PLATFORM --region=REGION --stage=STAGE
```

### Arguments:
* archive_path: The file path or s3 that contains BentoML bundles.

### Options:
* platform: REQUIRED.  The platform that you want to deploy bentoml bundle to.  For serverless, we support aws-lambda, aws-lambda-py2 and gcp-function.
* region: OPTIONAL The cloud provider's region you want to deploy in.
* stage: OPTIONAL Stage is a helper value that identify the position in the workflow process. DEFAULT value is dev

In [ ]:
!bentoml check-deployment-status ./model --platform=aws-sagemaker --region=us-west-2

# Delete serverless Deployment


```bash
bentoml delete-deployment ARCHIVE_PATH --platform=PLATFORM
```
### Arguments:
* archive_path: The file path or s3 that contains BentoML bundles.

### Options:
* platform: REQUIRED.  The platform that you want to deploy bentoml bundle to.  For serverless, we support aws-lambda, aws-lambda-py2 and gcp-function.
* region: OPTIONAL The cloud provider's region you want to deploy in.
* stage: OPTIONAL Stage is a helper value that identify the position in the workflow process. DEFAULT value is dev



In [ ]:
!bentoml delete-deployment ./model --platform aws-sagemaker --region us-west-2